In [18]:
import nltk
import math
from decimal import *
import numpy as np

In [19]:
tag_list = set()
word_set = set()
transition_dict = {}
tag_count = {}
corpus = nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]

In [20]:
# QUESTION 1
def gen_matrices(corpus):
    # Generate counts of all values
    for sentence in corpus:
        previous = "start"
        for data in sentence:
            word_set.add(data[0].lower())
            tag = data[1]
            tag_list.add(tag)

            if tag in tag_count: tag_count[tag] += 1
            else: tag_count[tag] = 1

            if (previous, tag) in transition_dict: transition_dict[(previous, tag)] += 1
            else: transition_dict[(previous, tag)] = 1
            
            previous = tag

    word_map['start'] = 0
    word_map['undefined'] = len(word_map)

    # Generate mappings
    idx = 1
    for word in word_set:
        word_map[word] = i
        idx += 1

    idx = 1
    tag_map['start'] = 0
    for tag in tag_list:
        tag_map[tag] = idx
        idx += 1

    # Generate probabilities of trans
    probabilities = {}
    for key in transition_dict:
        den = 0
        val = key[0]
        for key_2 in transition_dict:
            if key_2[0] == val:
                den += count_dict[key_2]
        probabilities[key] = Decimal(count_dict[key])/(den)

    for tag in tag_list:
        if ("start", tag) not in probabilities:
            probabilities[("start", tag)] = Decimal(
                1) / Decimal(len(word_set) + tag_count[tag])
    for tag1 in tag_list:
        for tag2 in tag_list:
            if (tag1, tag2) not in probabilities:
                probabilities[(tag1, tag2)] = Decimal(
                    1) / Decimal(len(word_set) + tag_count[tag1])
    count_word = {}
    for value in corpus:
        for data in value:
            word = data[0]
            tag = data[1]
            if (word, tag) in count_word:
                count_word[(word, tag)] += 1
            else:
                count_word[(word, tag)] = 1

    # Generate probs of observations
    obs_probs = {}
    for key in count_word:
        obs_probs[key] = Decimal(count_word[key])/tag_count[key[1]]

    transition_matrix = np.zeros((len(tag_map), len(tag_map)))

    for key, value in transition_prob.items():
        i = tag_map[key[0]]
        j = tag_map[key[1]]
        transition_matrix[i, j] = value

    ob_matrix = np.zeros((len(tag_map), len(word_map)+1))

    for key, value in obs_probs.items():
        i = tag_map[key[1]]
        j = word_map[key[0].lower()]

        obs_matrix[i][j] = value
    random = 1/len(tag_map)
    for tag in tag_map.keys():
        obs_matrix[tag_map[tag]][word_map['undefined']] = random

    return transition_matrix[0], transition_matrix, obs_matrix

In [21]:
def viterbi(obs, pi, a, b):
    num_states = len(b)
    num_obs = len(obs)


    path = np.zeros(T, dtype=int)
    delta = np.zeros((num_states, num_obs))
    phi = np.zeros((num_states, num_obs))
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0
    
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]]
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])

    path[T-1] = np.argmax(delta[:, T-1])

    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]

    return path

In [22]:
def gen_sent(sentence):
    ints = []
    ints.append(0) # for start
    for tag in sentence:
        word = tag[0]
        try:
            idx = word_map[word]
        except:
            idx = word_map['undefined']
        ints.append(idx)
    return ints

In [23]:
def test_run():
    train_data = init_data()
    pi, a, b = load_matricies(train_data)
    test = nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10153]
    for t in test:
        print(t)
        idx_sentence = build_sentence(t)
        vt = viterbi(idx_sentence, pi, a, b)   
        final = []
        for state in states:
            for tag in tag_map.keys():
                if tag_map[tag] == state:
                    final.append(tag)
        return final[1:]

In [24]:
test_run()

TypeError: 'NoneType' object is not iterable